In [25]:
import os
import pandas as pd
import sys
import torch

sys.path.append("../")
# sys.path.append("../hms_pipeline")

from hms_pipeline.spec_dataset import SpecDataset, infer_spec_dataset
from hms_pipeline.eeg_model import SpecModel, SpecVitModel
from hms_pipeline.kaggle_kl_div import eval_subm, eval_subm_g10

from hms_pipeline.eeg_model import EegInferModule


In [26]:
torch.set_float32_matmul_precision("medium")

In [27]:

model_dir = "/home/maxc/workspace/kaggle-hms/models-hmsv6-1/hmsv6-maxvits-4imgs-w025-kl1-512"
model_dir = "/home/maxc/workspace/kaggle-hms/models-hmsv6-1/hmsv6-convnexts-4imgs-w025-kl0-512"
model_dir = "/home/maxc/workspace/kaggle-hms/models-hmsv6-1/hmsv6-maxvitt-4imgs-w025-kl1-avgmax"
model_dir = "/home/maxc/workspace/kaggle-hms/models-hmsv6-1/hmsv6-convnexts-w025-3imgs-kl1-bandclip-20s"
model_dir = "/home/maxc/workspace/kaggle-hms/models-hmsv6/hmsv6-maxvitt-w025-3imgs-kl1-bandclip-avgmax"
model_dir =  "/home/maxc/workspace/kaggle-hms/models-hmsv6/hmsv6-maxvits-w025-3imgs-kl1-bandclip"

model_dir = "/home/maxc/workspace/kaggle-hms/models-hmsv6-1/hmsv6-convnexts-w025-4imgs-kl1-bandclip-12s-large"

model_dir = "/home/maxc/workspace/kaggle-hms/models-hmsv6-1/hmsv6-maxvitt-w025-4imgs-kl1-bandclip-lrf16-12s"
output_dir = "/home/maxc/workspace/kaggle-hms/oof_results"



lr_flip = False

# output csv_name 

if lr_flip:
    csv_name = "lrflip_" + model_dir.split("/")[-1] + ".csv"
    output_csv_path = os.path.join(output_dir, csv_name)
    output_logit_csv_name = "lrflip_" + model_dir.split("/")[-1] + "_logit.csv"
    output_logit_csv_path = os.path.join(output_dir, output_logit_csv_name)
else:
    csv_name = model_dir.split("/")[-1] + ".csv"
    output_csv_path = os.path.join(output_dir, csv_name)
    output_logit_csv_name = model_dir.split("/")[-1] + "_logit.csv"
    output_logit_csv_path = os.path.join(output_dir, output_logit_csv_name)
print(f"output_csv_path: {output_csv_path}")
print(f"output_logit_csv_path: {output_logit_csv_path}")

infer_dataset_group = "unique_id"
# infer_dataset_group = "eeg_id"

output_csv_path: /home/maxc/workspace/kaggle-hms/oof_results/hmsv6-maxvitt-w025-4imgs-kl1-bandclip-lrf16-12s.csv
output_logit_csv_path: /home/maxc/workspace/kaggle-hms/oof_results/hmsv6-maxvitt-w025-4imgs-kl1-bandclip-lrf16-12s_logit.csv


In [28]:
def get_config_path(model_dir):
    # get the file starting with hms-config
    configs = os.listdir(model_dir)
    config_path = [config for config in configs if config.startswith("hms-config")][0]
    return os.path.join(model_dir, config_path)

def get_best_ckpts(model_dir):
    ckpts = os.listdir(model_dir)
    ckpts = [ckpt for ckpt in ckpts if ckpt.endswith(".ckpt")]
    #sort ckpts by fold
    ckpts = sorted(ckpts)
    # all the check points startes with model-f{fold}-val_g10_score={g10_score}-val_score={val_score}.ckpt
    # we want to keep only the best ckpt with min g10_score for each fold
    best_ckpts = {}
    for ckpt in ckpts:
        fold = ckpt.split("-")[1]
        fold_num = int(fold.split("f")[1])
        g10_score_str = ckpt.split("-")[2]
        # print(g10_score_str)
        g10_score = float(g10_score_str.split("=")[1])
        
        if fold_num not in best_ckpts:
            best_ckpts[fold_num] = ckpt
        else:
            if g10_score < float(best_ckpts[fold_num].split("-")[2].split("=")[1]):
                best_ckpts[fold_num] = ckpt

    return best_ckpts

In [29]:
best_ckpts = get_best_ckpts(model_dir)
print(best_ckpts)
config_path = get_config_path(model_dir)
print(config_path)

{0: 'model-f0-val_g10_score=0.3130-val_score=0.5258.ckpt', 1: 'model-f1-val_g10_score=0.2940-val_score=0.4809.ckpt', 2: 'model-f2-val_g10_score=0.3218-val_score=0.5256.ckpt', 3: 'model-f3-val_g10_score=0.2954-val_score=0.5110.ckpt', 4: 'model-f4-val_g10_score=0.2876-val_score=0.4826.ckpt'}
/home/maxc/workspace/kaggle-hms/models-hmsv6-1/hmsv6-maxvitt-w025-4imgs-kl1-bandclip-lrf16-12s/hms-configs.yaml


In [30]:
folds_dir = "/home/maxc/workspace/kaggle-hms/folds"

import yaml
import matplotlib.pyplot as plt

with open(config_path, "r") as f:
        model_config = yaml.load(f, Loader=yaml.FullLoader)
print(model_config)

model_config["val_dataset_group"] = infer_dataset_group

if lr_flip:
    model_config["val_aug_probs"]["lrflip_prob"] = 1.0

all_result_dfs = []
all_result_logits_dfs = []
for fold, ckpt in best_ckpts.items():
    val_df_path = os.path.join(folds_dir, f"val_fold_{fold}.csv")
    val_df = pd.read_csv(val_df_path)
    
        
    infer_ds = SpecDataset(val_df,
                    model_config,
                    train = False,
                )
    print(len(infer_ds))

    ckpt_path = os.path.join(model_dir, ckpt)

    if model_config["model_type"] == "SpecVitModel":
        print("Using SpecVitModel")
        model = SpecVitModel(model_config["spec_backbone"], vit_model = model_config["vit_model"], img_size = model_config["img_size"], pretrained=False, dropout=model_config["dropout"])
    else:
        print("Using SpecModel")
        model = SpecModel(model_config["spec_backbone"], global_pool = model_config["global_pool"],  pretrained = False, img_size = model_config["img_size"])

    # model_config["use_ema"] = False
    module = EegInferModule(model, model_config)
    ckpt = torch.load(ckpt_path)
    module.load_state_dict(ckpt['state_dict'], strict=False)
    result_df, result_logits_df = infer_spec_dataset(eeg_dataset=infer_ds, model = module, batch_size=32)
    all_result_dfs.append(result_df)
    all_result_logits_dfs.append(result_logits_df)

{'trainer': {'max_epochs': 10, 'min_epochs': 5, 'enable_progress_bar': True, 'devices': 1, 'deterministic': False, 'precision': '16'}, 'patience': 3, 'train_bs': 14, 'valid_bs': 14, 'workers': 8, 'wandb_project': 'hms_v6', 'output_dir': '../models-hmsv6-1/hmsv6-maxvitt-w025-4imgs-kl1-bandclip-lrf16-12s', 'train_folds': [0, 1, 2, 3, 4], 'train_dataset_group': 'unique_label', 'val_dataset_group': 'eeg_id', 'raw_eeg_dir': '/home/maxc/workspace/kaggle-hms/data/v6/raw_eegs_sosbandclip_2500', 'eeg_spec_dir': '/home/maxc/workspace/kaggle-hms/data/v6/eeg_specs_h100w250gf5fft1024wl200lc05_norm_const', 'long_spec_dir': '/home/maxc/workspace/kaggle-hms/data/v6/long_specs', 'l7_weight': 0.25, 'inverse_kl_weight': True, 'inverse_kl_mean': [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666], 'kl_multiplier': 1.0, 'vote_weight': False, 'img_types': ['long_spec', 'eeg_spec', 'full_raw_eeg', 'raw_eeg'], 'signals': ['LL', 'RL', '

In [ ]:
result_df = pd.concat(all_result_dfs)
result_df.to_csv(output_csv_path, index=False)
result_df.head()

,eeg_id,eeg_sub_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1001487592,0,0.009771,0.184658,0.772392,0.001854,0.006188,0.025136
1,1001487592,1,0.012339,0.148768,0.813579,0.001241,0.004129,0.019944
2,1001487592,2,0.011068,0.197854,0.766643,0.001297,0.003965,0.019173
3,1001487592,3,0.010702,0.279821,0.661342,0.002435,0.008383,0.037316
4,1003458521,0,0.614065,0.125151,0.001935,0.092849,0.005244,0.160755


In [ ]:
result_logits_df = pd.concat(all_result_logits_dfs)
result_logits_df.to_csv(output_logit_csv_path, index=False)
result_logits_df.head()

,eeg_id,eeg_sub_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1001487592,0,-0.834580,2.104474,3.535462,-2.496653,-1.291346,0.110276
1,1001487592,1,-0.450730,2.038898,3.737956,-2.747182,-1.545426,0.029438
2,1001487592,2,-0.563513,2.319951,3.674443,-2.707443,-1.590178,-0.014069
3,1001487592,3,-0.956949,2.306753,3.166873,-2.437598,-1.201162,0.292033
4,1003458521,0,2.746888,1.156308,-3.013068,0.857767,-2.016048,1.406671


In [ ]:
gt_df_path = "/home/maxc/workspace/kaggle-hms/folds/all_targets.csv"
gt_df = pd.read_csv(gt_df_path)

print(eval_subm(result_df,gt_df))
print(eval_subm_g10(result_df,gt_df))

(106800, 7) (106800, 7) (106800, 7)
0.5962674836693916
(106800, 7) (39946, 7) (39946, 7)
0.283454511485884


In [ ]:
output_path = os.path.join(model_dir, "subm/oof_result.csv")
logits_output_path = os.path.join(model_dir, "subm/oof_result_logits.csv")
result_df.to_csv(output_path, index=False)
result_logits_df.to_csv(logits_output_path, index=False)


In [ ]:
first_sample = result_df.groupby("eeg_id").first().reset_index()
print(eval_subm(first_sample,gt_df))
print(eval_subm_g10(first_sample,gt_df))

(106800, 7) (17089, 7) (17089, 7)
0.505232635725143
(106800, 7) (5441, 7) (5441, 7)
0.24427100009129857


In [ ]:
# group by eeg id and keep the center sample
center_sample = result_df.groupby("eeg_id").nth(1).reset_index()
